In [1]:
import pandas as pd

In [2]:
medicare_telehealth = pd.read_csv('./data/Medicare_Telehealth_Trends_Q2_2024.csv')

telemedicine_use = pd.read_csv('./data/Telemedicine_Use_in_the_Last_4_Weeks.csv')

In [3]:
# Drop unnecessary columns
telemedicine_reduced = telemedicine_use.drop(columns=[
    'Indicator', 'Phase', 'Confidence Interval', 'Quartile Range', 'Suppression Flag'
])

medicare_reduced = medicare_telehealth.drop(columns=[
    'Bene_Mdcr_Entlmt_Stus', 'Total_PartB_Enrl'
])

# Drop na value
telemedicine_final = telemedicine_reduced.dropna()
medicare_final = medicare_reduced.dropna()

# Save the cleaned datasets into /data directory with id
telemedicine_final.to_csv('./data/telemedicine_cleaned.csv', index_label='ID')
medicare_final.to_csv('./data/medicare_cleaned.csv', index_label='ID')

In [4]:
# Splits the telemedicine dataset by 'Group' and saves each group as a separate CSV file.
# Only keep and generate CSV files for specific groups that are needed
# These are: 'National Estimate', 'By Age', 'By State'
groups = ['National Estimate', 'By Age', 'By State']

datasets = {group: telemedicine_final[telemedicine_final['Group'] == group] for group in groups}

saved_files = []
for group, df in datasets.items():
    save_group_name = group.replace('/', '_').replace(' ', '_').replace('?', '').replace(':', '')
    filename = f'./data/telemedicine_group_{save_group_name}.csv'
    df.to_csv(filename, index=False)
    saved_files.append(filename)

saved_files

['./data/telemedicine_group_National_Estimate.csv',
 './data/telemedicine_group_By_Age.csv',
 './data/telemedicine_group_By_State.csv']

In [5]:
# Convert cleaned telemedicine CSV files to sql

telemedicine_national_estimate = pd.read_csv('./data/telemedicine_group_National_Estimate.csv')
telemedicine_by_age = pd.read_csv('./data/telemedicine_group_By_Age.csv')
telemedicine_by_state = pd.read_csv('./data/telemedicine_group_By_State.csv')

telemedicine_datasets = {
    "telemedicine_national_estimate": telemedicine_national_estimate,
    "telemedicine_by_age": telemedicine_by_age,
    "telemedicine_by_state": telemedicine_by_state
}

# Function to assign correct SQL type
def get_sql_type_telemedicine(col):
    if col == 'Time Period':
        return "INTEGER"
    elif col == 'Value':
        return "FLOAT"
    elif col in ['Low CI', 'High CI']:
        return "FLOAT"
    elif col in ['Time Period Start Date', 'Time Period End Date']:
        return "DATE"
    else:
        return "TEXT"

# Function to convert date format MM/DD/YYYY to YYYY-MM-DD
def convert_date_format(date_str):
    if pd.isnull(date_str):
        return None
    try:
        return pd.to_datetime(date_str, format='%m/%d/%Y').strftime('%Y-%m-%d')
    except ValueError:
        return date_str 

sql_statements = ""

# Process each dataset to create SQL statements
for table_name, df in telemedicine_datasets.items():
    columns = ["ID INTEGER"]
    for col in df.columns:
        col_type = get_sql_type_telemedicine(col)
        columns.append(f"{col} {col_type}")
    
    indent = "    "
    create_table = f"CREATE TABLE {table_name} (\n" + ",\n".join([indent + col for col in columns]) + "\n);\n\n"
    sql_statements += create_table

    # INSERT INTO statements
    for i, (_, row) in enumerate(df.iterrows()):
        values = [str(i)]
        for col in df.columns:
            val = row[col]
            if col in ['Time Period Start Date', 'Time Period End Date'] and pd.notnull(val):
                val = convert_date_format(val)
            if pd.isnull(val):
                values.append("NULL")
            elif get_sql_type_telemedicine(col) in ["FLOAT", "INTEGER"]:
                values.append(str(val))
            elif get_sql_type_telemedicine(col) == "DATE":
                values.append(f"'{val}'")
            else:  # TEXT
                safe_val = str(val).replace("'", "''")
                values.append(f"'{safe_val}'")
        insert_stmt = f"INSERT INTO {table_name} VALUES ({', '.join(values)});\n"
        sql_statements += insert_stmt
    sql_statements += "\n"

# Save SQL statements to file
sql_file_path = "./data/telemedicine_groups.sql"
with open(sql_file_path, "w", encoding='utf-8') as f:
    f.write(sql_statements)

sql_file_path

'./data/telemedicine_groups.sql'

In [6]:
# Convert medicare_cleaned.csv to sql
medicare_cleaned_df = pd.read_csv('./data/medicare_cleaned.csv')

# Function to assign correct SQL type
def get_sql_type_medicare(col):
    if col in ['ID', 'Year', 'quarter', 'Total_Bene_TH_Elig', 'Total_Bene_Telehealth']:
        return "INTEGER"
    elif col == 'Pct_Telehealth':
        return "FLOAT"
    else:
        return "TEXT"

sql_statements = ""

table_name = "medicare_telehealth_cleaned"

# Process each dataset to create SQL statements
columns = [f"{col} {get_sql_type_medicare(col)}" for col in medicare_cleaned_df.columns]

indent = "    "
create_table = f"CREATE TABLE {table_name} (\n" + ",\n".join([indent + col for col in columns]) + "\n);\n\n"
sql_statements += create_table

# INSERT INTO statements
for i, (_, row) in enumerate(medicare_cleaned_df.iterrows()):
    values = []
    for col in medicare_cleaned_df.columns:
        val = row[col]
        if pd.isnull(val):
            values.append("NULL")
        elif get_sql_type_medicare(col) in ["INTEGER", "FLOAT"]:
            values.append(str(val))
        else:  # TEXT
            safe_val = str(val).replace("'", "''")
            values.append(f"'{safe_val}'")
    insert_stmt = f"INSERT INTO {table_name} VALUES ({', '.join(values)});\n"
    sql_statements += insert_stmt

# Save to SQL file
sql_file_path = './data/medicare_cleaned.sql'
with open(sql_file_path, 'w', encoding='utf-8') as f:
    f.write(sql_statements)

sql_file_path

'./data/medicare_cleaned.sql'